In [101]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

In [102]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/train_final.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/test_final.csv')

In [ ]:
pd.options.display.max_info_columns = 200
train_dataset.info()
train_dataset.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 146 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   continuous_annual_inc                   50000 non-null  float64
 1   continuous_annual_inc_joint             220 non-null    float64
 2   continuous_delinq_2yrs                  50000 non-null  float64
 3   continuous_dti                          49999 non-null  float64
 4   continuous_dti_joint                    220 non-null    float64
 5   continuous_fico_range_high              50000 non-null  float64
 6   continuous_fico_range_low               50000 non-null  float64
 7   continuous_funded_amnt                  50000 non-null  float64
 8   continuous_funded_amnt_inv              50000 non-null  float64
 9   continuous_inq_last_6mths               50000 non-null  float64
 10  continuous_installment                  50000 non-null  f

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,...,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
count,5.000000e+04,220.000000,50000.00000,49999.000000,220.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,26083.000000,15052.000000,9495.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,7.835886e+04,109248.802545,0.34738,19.080239,18.831864,698.404460,694.404300,14332.53650,14325.533000,0.620820,428.892109,11.986125,675.644860,658.428400,14332.53650,0.795760,34.197485,44.214589,65.500790,11.944960,0.252820,0.033880,0.002000,0.038880,0.033380,0.018000,0.012260,0.004400,0.02756,0.137380,0.028700,0.025140,0.016960,0.023600,0.08170,0.083120,0.008160,0.005800,0.012500,0.019380,...,0.006040,0.000520,1.0,0.056600,0.065920,0.067100,0.005860,0.055120,0.061300,0.056820,0.030020,0.064140,0.059300,0.015560,0.036840,0.012960,0.041540,0.044880,0.02210,0.003780,0.036220,0.066360,0.023940,0.017880,0.05160,0.004620,0.009440,0.027920,0.016740,0.002320,0.008180,0.028180,0

In [ ]:
test_dataset.info()
test_dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 146 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   continuous_annual_inc                   50000 non-null  float64
 1   continuous_annual_inc_joint             181 non-null    float64
 2   continuous_delinq_2yrs                  50000 non-null  float64
 3   continuous_dti                          49999 non-null  float64
 4   continuous_dti_joint                    180 non-null    float64
 5   continuous_fico_range_high              50000 non-null  float64
 6   continuous_fico_range_low               50000 non-null  float64
 7   continuous_funded_amnt                  50000 non-null  float64
 8   continuous_funded_amnt_inv              50000 non-null  float64
 9   continuous_inq_last_6mths               50000 non-null  float64
 10  continuous_installment                  50000 non-null  f

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,...,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_grade_29_one_hot,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot
count,5.000000e+04,181.000000,50000.000000,49999.000000,180.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,25927.000000,14934.000000,8868.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,7.757624e+04,107612.666354,0.356780,18.926104,17.380222,698.500620,694.50050,14525.736500,14516.862882,0.578180,434.119275,11.965563,677.453720,660.375800,14525.736500,0.804520,34.184055,44.081626,65.194632,11.832640,0.237560,0.035040,0.002000,0.040300,0.033820,0.017860,0.011100,0.003640,0.027760,0.136200,0.028020,0.023380,0.016320,0.024120,0.083240,0.084780,0.009020,0.005340,0.013140,0.020800,...,0.005860,0.000700,1.0,0.056840,0.061260,0.064860,0.00618,0.057120,0.062240,0.060700,0.030920,0.062820,0.060240,0.016300,0.039760,0.013500,0.042500,0.040640,0.021580,0.002580,0.03320,0.06594,0.022900,0.018600,0.056640,0.003940,0.010220,0.027160,0.01770,0.001640,0.007280,0.0290

In [ ]:
print(train_dataset['loan_status'].value_counts())
print(test_dataset['loan_status'].value_counts())
print(train_dataset.drop('loan_status',axis=1).shape)

1    39788
0    10212
Name: loan_status, dtype: int64
1    40226
0     9774
Name: loan_status, dtype: int64
(50000, 145)


In [ ]:

import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    



In [ ]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")


In [ ]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [ ]:
fitter = LGBFitter(label='loan_status')

In [ ]:
params = {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [ ]:
train_pred, test_pred, acc_result, models = fitter.train_k_fold(kfold, train_dataset, test_dataset, params = params)

The minimum is attained in round 413
The minimum is attained in round 578
The minimum is attained in round 397
The minimum is attained in round 623
The minimum is attained in round 329


基准，验证集上的表现

In [ ]:
acc_result

[0.07179999999999997,
 0.08140000000000003,
 0.08330000000000004,
 0.0837,
 0.08089999999999997]

第一个模型最佳，选择第一个跑测试集

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = models[0].predict(test_dataset.drop(columns='loan_status'))
print('The mean_squared_error: %f'% (mean_squared_error(test_dataset['loan_status'], y_pred)))

The mean_squared_error: 0.059483


In [ ]:
#变量的重要性
print(len(models))
model_lgb=models[0]
feature_importance = pd.DataFrame({'feature_name': model_lgb.feature_name(),'importance': model_lgb.feature_importance()})
feature_importance.sort_values( ascending=False, inplace=True,by=['importance'])
print(feature_importance[:20])

5
                               feature_name  importance
12          continuous_last_fico_range_high        1505
3                            continuous_dti         914
0                     continuous_annual_inc         691
10                   continuous_installment         680
15        continuous_mths_since_last_delinq         611
5                continuous_fico_range_high         500
18                      continuous_open_acc         463
11                      continuous_int_rate         436
7                    continuous_funded_amnt         408
16   continuous_mths_since_last_major_derog         389
17        continuous_mths_since_last_record         377
143                 discrete_term_1_one_hot         334
13           continuous_last_fico_range_low         307
8                continuous_funded_amnt_inv         307
82           discrete_emp_length_12_one_hot         260
9                 continuous_inq_last_6mths         189
6                 continuous_fico_range_low   

In [ ]:
#取前20个特征构建数据集 

important_cols = []
for c in feature_importance[:20]['feature_name']:
  important_cols.append(c)

important_cols.append('loan_status')

In [ ]:
len(important_cols)

21

In [ ]:
train_dataset1,test_dataset1 = train_dataset[important_cols],test_dataset[important_cols]

train_pred, test_pred, acc_result, models = fitter.train_k_fold(kfold, train_dataset1,test_dataset1, params = params)

The minimum is attained in round 444
The minimum is attained in round 370
The minimum is attained in round 378
The minimum is attained in round 505
The minimum is attained in round 379


In [ ]:
acc_result

[0.07250000000000001, 0.0807, 0.08460000000000001, 0.0857, 0.0817]

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = models[0].predict(test_dataset1.drop(columns='loan_status'))
print('The mean_squared_error: %f'% (mean_squared_error(test_dataset1['loan_status'], y_pred)))

The mean_squared_error: 0.060102


# Baseline:   The mean_squared_error: 0.059483
# 前50个特征： The mean_squared_error: 0.059531
# 前30个特征： The mean_squared_error: 0.059532
# 前20个特征： The mean_squared_error: 0.059418
# 前10个特征： The mean_squared_error: 0.060102

In [ ]:
#选择重要性前20个变量，来构造延伸变量
feature_importance[:20]

,feature_name,importance
12,continuous_last_fico_range_high,1505
3,continuous_dti,914
0,continuous_annual_inc,691
10,continuous_installment,680
15,continuous_mths_since_last_delinq,611
5,continuous_fico_range_high,500
18,continuous_open_acc,463
11,continuous_int_rate,436
7,continuous_funded_amnt,408
16,continuous_mths_since_last_major_derog,389


In [119]:
#构造月还款能力 monthly_installment_capacity=continuous_annual_inc/12-continuous_installment

train_dataset20,test_dataset20 = train_dataset[important_cols],test_dataset[important_cols]

train_dataset20['monthly_installment_capacity']= (train_dataset20['continuous_annual_inc'] / 12) - train_dataset20['continuous_installment']
test_dataset20['monthly_installment_capacity'] = (test_dataset20['continuous_annual_inc'] / 12) - test_dataset20['continuous_installment']

print(train_dataset20.shape)
print(test_dataset20.shape)
train_dataset20.head()



(50000, 22)
(50000, 22)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,continuous_last_fico_range_high,continuous_dti,continuous_annual_inc,continuous_installment,continuous_mths_since_last_delinq,continuous_fico_range_high,continuous_open_acc,continuous_int_rate,continuous_funded_amnt,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,discrete_term_1_one_hot,continuous_last_fico_range_low,continuous_funded_amnt_inv,discrete_emp_length_12_one_hot,continuous_inq_last_6mths,continuous_fico_range_low,continuous_delinq_2yrs,discrete_home_ownership_1_one_hot,discrete_emp_length_1_one_hot,loan_status,monthly_installment_capacity
0,564.0,5.91,55000.0,123.03,30.0,679.0,7.0,13.99,3600.0,30.0,NaN,1,560.0,3600.0,0,1.0,675.0,0.0,1,1,1,4460.303333
1,699.0,16.06,65000.0,820.28,6.0,719.0,22.0,11.99,24700.0,NaN,NaN,1,695.0,24700.0,0,4.0,715.0,1.0,1,1,1,4596.386667
2,704.0,10.78,63000.0,432.66,NaN,699.0,6.0,10.78,20000.0,NaN,NaN,0,700.0,20000.0,0,0.0,695.0,0.0,1,1,1,4817.340000
3,704.0,25.37,104433.0,289.91,12.0,699.0,12.0,22.45,10400.0,NaN,NaN,0,700.0,10400.0,0,3.0,695.0,1.0,1,0,1,8412.840000
4,759.0,10.20,34000.0,405.18,NaN,694.0,5.0,13.44,11950.0,NaN,NaN,1,755.0,11950.0,0,0.0,690.0,0.0,0,0,1,2428.153333


In [114]:
train_pred, test_pred, acc_result, models = fitter.train_k_fold(kfold, train_dataset20,test_dataset20, params = params)

The minimum is attained in round 535
The minimum is attained in round 391
The minimum is attained in round 388
The minimum is attained in round 540
The minimum is attained in round 310


In [115]:
acc_result

[0.07179999999999997,
 0.08199999999999996,
 0.08430000000000004,
 0.08379999999999999,
 0.07979999999999998]

In [118]:
y_pred = models[0].predict(test_dataset20.drop(columns='loan_status'))
print('The mean_squared_error: %f'% (mean_squared_error(test_dataset20['loan_status'], y_pred)))

The mean_squared_error: 0.059455


In [125]:
### 构造变量：信用分平方 squared_continuous_last_fico_range_high 

train_dataset20,test_dataset20 = train_dataset[important_cols],test_dataset[important_cols]

train_dataset20['squared_continuous_last_fico_range_high']= train_dataset20['continuous_last_fico_range_high']**2
test_dataset20['squared_continuous_last_fico_range_high'] = test_dataset20['continuous_last_fico_range_high']**2

print(train_dataset20.shape)
print(test_dataset20.shape)
train_dataset20.head()

train_dataset20.info()



(50000, 22)
(50000, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   continuous_last_fico_range_high          50000 non-null  float64
 1   continuous_dti                           49999 non-null  float64
 2   continuous_annual_inc                    50000 non-null  float64
 3   continuous_installment                   50000 non-null  float64
 4   continuous_mths_since_last_delinq        26083 non-null  float64
 5   continuous_fico_range_high               50000 non-null  float64
 6   continuous_open_acc                      50000 non-null  float64
 7   continuous_int_rate                      50000 non-null  float64
 8   continuous_funded_amnt                   50000 non-null  float64
 9   continuous_mths_since_last_major_derog   15052 non-null  float64
 10  continuous_mths_since_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
train_pred, test_pred, acc_result, models = fitter.train_k_fold(kfold, train_dataset20,test_dataset20, params = params)

The minimum is attained in round 498
The minimum is attained in round 455
The minimum is attained in round 380
The minimum is attained in round 704
The minimum is attained in round 352


In [127]:
acc_result

[0.07210000000000005,
 0.08179999999999998,
 0.08460000000000001,
 0.08360000000000001,
 0.08089999999999997]